# Introduction to numerical algorithms
## Practice class 4

In [2]:
import time
import numpy as np
import matplotlib.pyplot as plt

### Task 1
First use the algorithms written in class.

1. Use Gauss Elimination to solve the following equations.

$$\begin{gather}
3x_1−x_2+4x_3=2\\
17x_1+2x_2+x_3=14\\
x_1+12x_2−7x_3=54
\end{gather}$$
2. Write the Gauss-Jordan Elimination (**I mean full pivoting by it!**)to solve the above equations.
3. Find some clever way to visualize each step (eg. printin the augmented matrix or plotting a heatmap of the matrix at each step).
4. Get the lower triangular matrix $L$ and upper triangular matrix $U$ from the equations above.
5. Compare execution times for our vs numpy/scipy Gauss elimination! Go up with matrix sizes to at least $N=25$! To measure time:

```python
start = time.time()
# your code you want to time
stop = time.time()
tot_time = stop - start
```

Make a plot, where you plot the two lines representing the time needed for the different matrix sizes.

### Additional help

Here is a pseudo code that you can follow through to successfully implement the GE

```
Input: 
    A (n×n coefficient matrix)
    b (n×1 right-hand side vector)

Output:
    x (solution vector of length n)

Algorithm:

1. Form the augmented matrix B = [A | b].

2. Initialize column permutation record: col_perm = [0, 1, ..., n-1].

3. For k = 0 to n-1 do:
    
    a. Pivoting (choose best pivot):
        - Search the submatrix B[k:n, k:n] for the element with 
          the largest absolute value.
        - Let (i_max, j_max) be the position of this pivot.
        - If pivot value is 0 → the system has no unique solution.
    
    b. Swap rows if needed:
        - Swap row k with row i_max in B.

    c. Swap columns if needed:
        - Swap column k with column j_max in B.
        - Record this swap in col_perm (to undo later).
    
    d. Normalize pivot row:
        - Divide row k by the pivot value so that B[k, k] = 1.

    e. Eliminate all other entries in column k:
        - For each row i ≠ k:
            - Subtract (B[i, k] × row k) from row i.
        - This ensures that column k has zeros everywhere except at the pivot.

4. At this point, the left side of B is the identity matrix.
   The rightmost column contains the solution vector, but
   in permuted order (because of column swaps).

5. Undo column permutations:
    - Initialize x as a zero vector of length n.
    - For i = 0 to n-1:
        - Place solution entry b[i] into position col_perm[i] of x.

6. Return x.
```

To make the code modular, you can write functions tht perform only one subtast, eg. `swap_rows_and_cols()`, `add_rows_and_cols()`, `plot_state()` etc.

In [3]:
# Given the matrix A and vector b
A = np.array([[3,-1,4],
            [17,2,1],
            [1, 12, -7]])
b = np.array([2, 14, 54])

ind  = np.unravel_index(np.argmax(A), A.shape) # returns the index of the maximum value in the flattened array
# Algo to find the pivot:
print(A)
print(b)




[[ 3 -1  4]
 [17  2  1]
 [ 1 12 -7]]
[ 2 14 54]


In [4]:
def gauss_method(A, b):
    A = A.astype(float)
    b = b.astype(float)
    B = np.hstack((A, b.reshape(-1,1)))

    col_perm = np.arange(A.shape[1]) # column permutation vecto
    row_perm = np.arange(A.shape[0])
    n= A.shape[0]

    for k in range(A.shape[0]): # loop over each row
        sub = B[k:, k:] # submatrix B[k:n, k:n]
        pivot_idx = np.argmax(np.abs(B[k:, k:n])) # index of the max element in the submatrix
        col_idx = pivot_idx // A.shape[0] + k # column index of the pivot
        row_idx = pivot_idx % A.shape[0] + k # row index of the pivot

        if np.isclose(B[row_idx, col_idx], 0): 
            raise ValueError("Singular matrix") # check for zero pivot
        if row_idx != k: 
            B[[k, row_idx], :] = B[[row_idx, k], :] # swap rows in B
            row_perm[[k, row_idx]] = row_perm[[row_idx, k]] # update row permutation
        if col_idx != k:
            B[:, [k, col_idx]] = B[:, [col_idx, k]] # swap columns in B
            col_perm[[k, col_idx]] = col_perm[[col_idx, k]] # update column permutation 

        B[k] /= B[k,k] # normalize pivot row
        if k < n-1:
            B[k+1:] -= B[k+1: , k:k+1]*B[k] # eliminate below
        print(f"Step {k+1}:\n", B) # print the matrix after each step
        
    sol = np.zeros(n) # back substitution
    for i in range(n-1, -1, -1): 
        sol[i] = B[i, -1] - np.dot(B[i, i+1:n], sol[i+1:n]) # calculate solution
    sol = sol[col_perm.argsort()] # reorder solution according to column permutations
    return sol



In [5]:
gauss_method(A, b)



Step 1:
 [[ 1. -3. -4. -2.]
 [ 0. 23.  9. 18.]
 [ 0. 37. 41. 78.]]
Step 2:
 [[  1.          -4.          -3.          -2.        ]
 [  0.           1.           2.55555556   2.        ]
 [  0.           0.         -67.77777778  -4.        ]]
Step 3:
 [[ 1.         -4.         -3.         -2.        ]
 [ 0.          1.          2.55555556  2.        ]
 [-0.         -0.          1.          0.05901639]]


array([0.05901639, 5.57377049, 1.84918033])

In [6]:
#  Pivoting (choose best pivot):
''' Search the submatrix B[k:n, k:n] for the element with 
          the largest absolute value.
        - Let (i_max, j_max) be the position of this pivot.
        - If pivot value is 0 → the system has no unique solution.
'''

' Search the submatrix B[k:n, k:n] for the element with \n          the largest absolute value.\n        - Let (i_max, j_max) be the position of this pivot.\n        - If pivot value is 0 → the system has no unique solution.\n'

In [7]:
start = time.time()

gauss_method(A, b)
stop = time.time()
tot_time = stop - start
print("Time taken: ", tot_time)

Step 1:
 [[ 1. -3. -4. -2.]
 [ 0. 23.  9. 18.]
 [ 0. 37. 41. 78.]]
Step 2:
 [[  1.          -4.          -3.          -2.        ]
 [  0.           1.           2.55555556   2.        ]
 [  0.           0.         -67.77777778  -4.        ]]
Step 3:
 [[ 1.         -4.         -3.         -2.        ]
 [ 0.          1.          2.55555556  2.        ]
 [-0.         -0.          1.          0.05901639]]
Time taken:  0.004617929458618164


### Bonus

Have you ever wondered how scientific software computes a determinant? The formula that you learned for calculating determinants by hand is horribly cumbersome and computationally intractible for large matrices. This problem is meant to give you glimpse of what is actually going on under the hood.

If $A$ has an LU decomposition then $A=LU$. Use properties that you know about determinants to come up with a simple way to find the determinant for matrices that have an LU decomposition. Show all of your work in developing your formula (derive by hand in latex for eg.).

Once you have your formula for calculating $\det(A)$, write a Python function that accepts a matrix, produces the LU decomposition, and returns the determinant of $A$. 

Check your work against Python’s `np.linalg.det()` function.

### Pseudo code for partial pivoting LU decomposition
```
Input:
    A : square matrix (n × n)

Output:
    P : permutation matrix (n × n)
    L : lower triangular matrix (n × n, with 1s on diagonal)
    U : upper triangular matrix (n × n)

Algorithm:

1. Let n = number of rows in A.

2. Initialize:
    - U = copy of A
    - L = zero matrix of size n×n
    - P = identity matrix of size n×n

3. For i = 0 to n-1 do:
    
    a. Find pivot row:
        - Look at column i, rows i through n-1.
        - Find row index pivot with largest absolute value in this column.

    b. If pivot ≠ i:
        - Swap row i and row pivot in U.
        - Swap row i and row pivot in P.
        - If i > 0:
            Swap row i and row pivot in L, but only for columns 0 through i-1.

    c. Set L[i, i] = 1   (unit diagonal of L)

    d. For each row j from i+1 to n-1:
        - Compute multiplier: L[j, i] = U[j, i] / U[i, i]
        - Update row j of U: U[j, i:] = U[j, i:] - L[j, i] * U[i, i:]

4. Return P, L, U.
```

### Homework 1

Imagine that we have a $1$ meter long thin metal rod that has been heated to $100^\circ$ on the left-hand side and cooled to 0 on the right-hand side. We want to know the temperature every $10$ cm from left to right on the rod.

First we break the rod into equal $10$ cm increments. How many unknowns are there?

The temperature at each point along the rod is the average of the temperatures at the adjacent points. For example, if we let $T_1$ be the temperature at point $x_1$ then 

$$T_1=\frac{T_0+T_2}{2}.$$
 
1. Write a system of equations for each of the unknown temperatures.
2. Solve the system for the temperature at each unknown node using LU decomposition.
 
**BONUS:** Check other decompositions too! the most popular ones are Cholesky, QR, etc.

### Homework 2
For this problem you will be going to run a numerical experiment to see how the process of solving the equation 

$$Ax=b$$
using the LU factorization performs on random coefficient matrices $A$ and random right-hand sides $b$. You will compare against Python’s algorithm for solving linear systems.

Do the following:

1. Loop over the size of the matrix $n$.
2. Build a random matrix $A$ of size $n\times n$. You can do this with the code `A = np.matrix(np.random.randn(n,n))`
3. Build a random vector $b$ in $\mathbb{R}^n$. You can do this with the code `b = np.matrix(np.random.randn(n,1))`
4. Find Python’s answer to the problem $Ax=b$ using the command `exact = np.linalg.solve(A,b)`
5. Write code that uses your LU decomposition written in the practice class  to find a solution to the equation $Ax=b$.
6. Find the error between your answer and the exact answer using the code `np.linalg.norm(x - exact)`
7. Make a plot (`plt.semilogy()`) that shows how the error behaves as the size of the problem changes. You should run this for matrices of larger and larger size but be warned that the loop will run for quite a long time if you go above $300\times 300$ matrices. Just be patient.

In [9]:
A2 = np.matrix(np.random.rand(2,2))
A2

matrix([[0.32624465, 0.6820666 ],
        [0.5598601 , 0.85608422]])

In [11]:
b2 = np.matrix(np.random.rand(2, 1))
b2

matrix([[0.09044887],
        [0.87239252]])

In [12]:
exact = np.linalg.solve(A2, b2)
exact

matrix([[ 5.04633998],
        [-2.2811446 ]])